## UCI Adult Income Dataset - Data cleaning and preprocessing
This notebook is focused on the data preparation, cleaning and preprocessing for the UCI Adult Income Dataset

In [1]:
# import libraries
import pandas as pd 
import numpy as np
import os 

In [2]:
# Get working directory
current_dir= os.getcwd()
# Go one directory up to the root directory 
project_root_dir= os.path.dirname(current_dir)
data_dir = os.path.join(project_root_dir,'data')
raw_dir = os.path.join(data_dir,'raw')
processed_dir = os.path.join(data_dir,'processed')
# define path to result folder
result_dir=os.path.join(project_root_dir,'result')
# define path to docs folder
docs_dir=os.path.join(project_root_dir,'docs')

# create directory if they do not exist 
os.makedirs(raw_dir, exist_ok = True)
os.makedirs(processed_dir, exist_ok = True)
os.makedirs(result_dir, exist_ok = True)
os.makedirs(docs_dir, exist_ok = True)

## Read in the data

In [3]:
adult_data_filename = os.path.join(raw_dir,"adult.csv")
adult_df = pd.read_csv(adult_data_filename,header=None,na_values = '?', skipinitialspace = True)
adult_df.head(10)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K
5,37,Private,284582,Masters,14,Married-civ-spouse,Exec-managerial,Wife,White,Female,0,0,40,United-States,<=50K
6,49,Private,160187,9th,5,Married-spouse-absent,Other-service,Not-in-family,Black,Female,0,0,16,Jamaica,<=50K
7,52,Self-emp-not-inc,209642,HS-grad,9,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,45,United-States,>50K
8,31,Private,45781,Masters,14,Never-married,Prof-specialty,Not-in-family,White,Female,14084,0,50,United-States,>50K
9,42,Private,159449,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,5178,0,40,United-States,>50K


In [4]:
adult_df.shape

(32561, 15)

In [5]:
adult_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32561 entries, 0 to 32560
Data columns (total 15 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   0       32561 non-null  int64 
 1   1       30725 non-null  object
 2   2       32561 non-null  int64 
 3   3       32561 non-null  object
 4   4       32561 non-null  int64 
 5   5       32561 non-null  object
 6   6       30718 non-null  object
 7   7       32561 non-null  object
 8   8       32561 non-null  object
 9   9       32561 non-null  object
 10  10      32561 non-null  int64 
 11  11      32561 non-null  int64 
 12  12      32561 non-null  int64 
 13  13      31978 non-null  object
 14  14      32561 non-null  object
dtypes: int64(6), object(9)
memory usage: 3.7+ MB


## Data Cleaning
## 1Assign proper column names to the columns

In [6]:
adult_df.columns=["age", "workclass", "fnlwgt", "education", " education_num", "martial_status", "occupation", "relationship", "race", "sex", "capital_gain", "capital_loss", "hours_per_week", "native_country", "income"]
adult_df

,age,workclass,fnlwgt,education,education_num,martial_status,occupation,relationship,race,sex,capital_gain,capital_loss,hours_per_week,native_country,income
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32556,27,Private,257302,Assoc-acdm,12,Married-civ-spouse,Tech-support,Wife,White,Female,0,0,38,United-States,<=50K
32557,40,Private,154374,HS-grad,9,Married-civ-spouse,Machine-op-inspct,Husband,White,Male,0,0,40,United-States,>50K
32558,58,Private,151910,HS-grad,9,Widowed,Adm-clerical,Unmarried,White,Female,0,0,40,United-States,<=50K
32559,22,Private,201490,HS-grad,9,Never-married,Adm-clerical,Own-child,White,Male,0,0,20,United-States,<=50K


## Understanding the dataset
- age:Represents the age of the person, ranging from 1 to 90
- workclass: Represents the employment status of the person. Takes on the value: Federal_gov, lacal

In [8]:
np.unique(adult_df.workclass.to_list())

array(['Federal-gov', 'Local-gov', 'Never-worked', 'Private',
       'Self-emp-inc', 'Self-emp-not-inc', 'State-gov', 'Without-pay',
       'nan'], dtype='<U32')

In [10]:
np.unique(adult_df.fnlwgt.to_list())

array([  12285,   13769,   14878, ..., 1366120, 1455435, 1484705])

In [11]:
np.unique(adult_df.education.to_list())

array(['10th', '11th', '12th', '1st-4th', '5th-6th', '7th-8th', '9th',
       'Assoc-acdm', 'Assoc-voc', 'Bachelors', 'Doctorate', 'HS-grad',
       'Masters', 'Preschool', 'Prof-school', 'Some-college'],
      dtype='<U12')

## 2.Deal with missing values

In [7]:
adult_df.isnull().sum()

age                  0
workclass         1836
fnlwgt               0
education            0
 education_num       0
martial_status       0
occupation        1843
relationship         0
race                 0
sex                  0
capital_gain         0
capital_loss         0
hours_per_week       0
native_country     583
income               0
dtype: int64

In [106]:
adult_df['workclass']=adult_df['workclass'].fillna('Unknown')
adult_df['native_country']=adult_df['native_country'].fillna('Other')
adult_df['occupation']=adult_df['occupation'].fillna('Unknown')


KeyError: 'workclass'

In [8]:
adult_df.isnull().sum()

age                  0
workclass         1836
fnlwgt               0
education            0
 education_num       0
martial_status       0
occupation        1843
relationship         0
race                 0
sex                  0
capital_gain         0
capital_loss         0
hours_per_week       0
native_country     583
income               0
dtype: int64

## 3.Deal with Duplicate

In [9]:
adult_df.duplicated().sum()

24

In [10]:
adult_df[adult_df.duplicated(keep=False)]

,age,workclass,fnlwgt,education,education_num,martial_status,occupation,relationship,race,sex,capital_gain,capital_loss,hours_per_week,native_country,income
2303,90,Private,52386,Some-college,10,Never-married,Other-service,Not-in-family,Asian-Pac-Islander,Male,0,0,35,United-States,<=50K
3917,19,Private,251579,Some-college,10,Never-married,Other-service,Own-child,White,Male,0,0,14,United-States,<=50K
4325,25,Private,308144,Bachelors,13,Never-married,Craft-repair,Not-in-family,White,Male,0,0,40,Mexico,<=50K
4767,21,Private,250051,Some-college,10,Never-married,Prof-specialty,Own-child,White,Female,0,0,10,United-States,<=50K
4881,25,Private,308144,Bachelors,13,Never-married,Craft-repair,Not-in-family,White,Male,0,0,40,Mexico,<=50K
4940,38,Private,207202,HS-grad,9,Married-civ-spouse,Machine-op-inspct,Husband,White,Male,0,0,48,United-States,>50K
5104,90,Private,52386,Some-college,10,Never-married,Other-service,Not-in-family,Asian-Pac-Islander,Male,0,0,35,United-States,<=50K
5579,27,Private,255582,HS-grad,9,Never-married,Machine-op-inspct,Not-in-family,White,Female,0,0,40,United-States,<=50K
5805,20,Private,107658,Some-college,10,Never-married,Tech-support,Not-in-family,White,Female,0,0,10,United-States,<=50K
5842,25,Private,195994,1st-4th,2,Never-married,Priv-house-serv,Not-in-family,White,Female,0,0,40,Guatemala,<=50K


In [11]:
adult_df=adult_df.drop_duplicates()

In [13]:
adult_df.duplicated().sum()

0

In [14]:
adult_df.shape

(32537, 15)

## 4.Standardize Categorical variables

In [15]:
adult_df.dtypes == object

age               False
workclass          True
fnlwgt            False
education          True
 education_num    False
martial_status     True
occupation         True
relationship       True
race               True
sex                True
capital_gain      False
capital_loss      False
hours_per_week    False
native_country     True
income             True
dtype: bool

In [16]:
categorical_cols = adult_df.columns[adult_df.dtypes == object]
for col in categorical_cols:
    adult_df.loc[:, col] = adult_df[col].str.strip().str.lower() 
adult_df

,age,workclass,fnlwgt,education,education_num,martial_status,occupation,relationship,race,sex,capital_gain,capital_loss,hours_per_week,native_country,income
0,39,state-gov,77516,bachelors,13,never-married,adm-clerical,not-in-family,white,male,2174,0,40,united-states,<=50k
1,50,self-emp-not-inc,83311,bachelors,13,married-civ-spouse,exec-managerial,husband,white,male,0,0,13,united-states,<=50k
2,38,private,215646,hs-grad,9,divorced,handlers-cleaners,not-in-family,white,male,0,0,40,united-states,<=50k
3,53,private,234721,11th,7,married-civ-spouse,handlers-cleaners,husband,black,male,0,0,40,united-states,<=50k
4,28,private,338409,bachelors,13,married-civ-spouse,prof-specialty,wife,black,female,0,0,40,cuba,<=50k
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32556,27,private,257302,assoc-acdm,12,married-civ-spouse,tech-support,wife,white,female,0,0,38,united-states,<=50k
32557,40,private,154374,hs-grad,9,married-civ-spouse,machine-op-inspct,husband,white,male,0,0,40,united-states,>50k
32558,58,private,151910,hs-grad,9,widowed,adm-clerical,unmarried,white,female,0,0,40,united-states,<=50k
32559,22,private,201490,hs-grad,9,never-married,adm-clerical,own-child,white,male,0,0,20,united-states,<=50k


In [17]:
adult_df.columns

Index(['age', 'workclass', 'fnlwgt', 'education', ' education_num',
       'martial_status', 'occupation', 'relationship', 'race', 'sex',
       'capital_gain', 'capital_loss', 'hours_per_week', 'native_country',
       'income'],
      dtype='object')

## Re-code the workclass column

In [18]:
adult_df['workclass'].unique()

array(['state-gov', 'self-emp-not-inc', 'private', 'federal-gov',
       'local-gov', nan, 'self-emp-inc', 'without-pay', 'never-worked'],
      dtype=object)

In [78]:
adult_df.loc[:,'workclass'] = adult_df['workclass'].replace({
    'state-gov': 'government' ,
    'loc-gov': 'government' ,
    'federal-gov': 'government' ,
    'self-emp-not-inc': 'self-employed' ,
    'self-emp-inc': 'self-employed' ,
    'never-worked': 'unemployed' ,
    'without-pay': 'voluntary' ,
})

In [19]:
adult_df['workclass'].unique()

array(['state-gov', 'self-emp-not-inc', 'private', 'federal-gov',
       'local-gov', nan, 'self-emp-inc', 'without-pay', 'never-worked'],
      dtype=object)

## Re_code the education column

In [20]:
adult_df['education'].unique()

array(['bachelors', 'hs-grad', '11th', 'masters', '9th', 'some-college',
       'assoc-acdm', 'assoc-voc', '7th-8th', 'doctorate', 'prof-school',
       '5th-6th', '10th', '1st-4th', 'preschool', '12th'], dtype=object)

In [22]:
adult_df.loc[:,'education_level'] = adult_df['education'].map({
    'bachelors': 'tertiary',
    'masters': 'tertiary',
    'doctorate': 'tertiary',
    'prof-school': 'tertiary',
    'some-college': 'some college',
    'assoc-acdm': 'associate',
    'assoc-voc': 'associate',
    'hs-grad': 'high school graduate',
    '12th': 'secondary',
    '11th': 'secondary',
    '10th': 'secondary',
    '9th': 'secondary',
    '7th-8th': 'primary',
    '5th-6th': 'primary',
    '1st-4th': 'primary',
    'preschool': 'preschool'
})


In [24]:
adult_df.columns

Index(['age', 'workclass', 'fnlwgt', 'education', ' education_num',
       'martial_status', 'occupation', 'relationship', 'race', 'sex',
       'capital_gain', 'capital_loss', 'hours_per_week', 'native_country',
       'income', 'education_level'],
      dtype='object')

In [25]:
adult_df['education_level'].unique()

array(['tertiary', 'high school graduate', 'secondary', 'some college',
       'associate', 'primary', 'preschool'], dtype=object)

## Re-code the marital_status column

In [26]:
adult_df['martial_status'].unique()

array(['never-married', 'married-civ-spouse', 'divorced',
       'married-spouse-absent', 'separated', 'married-af-spouse',
       'widowed'], dtype=object)

In [27]:
adult_df.loc[:,'martial_status'] = adult_df['martial_status'].replace({
    'never-married': 'single' ,
    'married-civ-spouse': 'married' ,
   'married-spouse-absent': 'divorced or separated' ,
   'divorced': 'divorced or separated' ,
   'separated': 'divorced or separated' ,
   'married-af-spouse': 'married' 

})

In [28]:
adult_df['martial_status'].unique()

array(['single', 'married', 'divorced or separated', 'widowed'],
      dtype=object)

## Re-code the Occupation

In [29]:
adult_df['occupation'].unique()

array(['adm-clerical', 'exec-managerial', 'handlers-cleaners',
       'prof-specialty', 'other-service', 'sales', 'craft-repair',
       'transport-moving', 'farming-fishing', 'machine-op-inspct',
       'tech-support', nan, 'protective-serv', 'armed-forces',
       'priv-house-serv'], dtype=object)

In [31]:
adult_df.loc[:,'occupation_grouped'] = adult_df['occupation'].map({
    'adm-clerical': 'white collar',
    'exec-managerial': 'white collar',
    'handlers-cleaners': 'blue collar',
    'prof-specialty': 'white collar',
    'other-service': 'service',
    'sales': 'white collar',
    'craft-repair': 'blue collar',
    'transport-moving': 'blue collar',
    'farming-fishing': 'blue collar',
    'machine-op-inspct': 'blue collar',
    'tech-support': 'white collar',
    'protective-serv': 'service',
    'armed-forces': 'military',
    'priv-house-serv': 'service',
    'unknown': 'unknown'
})


In [86]:
adult_df.columns

Index(['age', 'workclass', 'fnlwgt', 'education', ' education_num',
       'martial_status', 'occupation', 'relationship', 'race', 'sex',
       'capital_gain', 'capital_loss', 'hours_per_week', 'native_country',
       'income', 'occupation_grouped'],
      dtype='object')

In [32]:
adult_df['occupation_grouped'].unique()

array(['white collar', 'blue collar', 'service', nan, 'military'],
      dtype=object)

## Re-code the relationship column

In [139]:
adult_df['relationship'].unique()

array(['not-in-family', 'husband', 'wife', 'own-child', 'unmarried',
       'other-relative'], dtype=object)

In [33]:
adult_df.loc[:,'relationship'] = adult_df['relationship'].replace({
    'wife': 'female spouse' ,
    'own-child': 'child' ,
    'not-in-family': 'single' ,
    'other-relative': 'extended relative' ,
    'unmarried': 'single' ,
    'husband': 'male spouse' 

})

In [34]:
adult_df['relationship'].unique()

array(['single', 'male spouse', 'female spouse', 'child',
       'extended relative'], dtype=object)

## Re-code the race column

In [35]:
adult_df['race'].unique()

array(['white', 'black', 'asian-pac-islander', 'amer-indian-eskimo',
       'other'], dtype=object)

In [36]:
adult_df.loc[:,'race'] = adult_df['race'].replace({
    'white': 'white' ,
    'asian-pac-islander': 'asian or pacific islander' ,
    'amer-indian-eskimo': 'american indian or eskimo' ,
    'other-relative': 'extended relative' ,
    'black': 'black',
    'other': 'other' 

})

In [37]:
adult_df['race'].unique()

array(['white', 'black', 'asian or pacific islander',
       'american indian or eskimo', 'other'], dtype=object)

## Re-code native_country column

In [38]:
adult_df['native_country'].unique()

array(['united-states', 'cuba', 'jamaica', 'india', nan, 'mexico',
       'south', 'puerto-rico', 'honduras', 'england', 'canada', 'germany',
       'iran', 'philippines', 'italy', 'poland', 'columbia', 'cambodia',
       'thailand', 'ecuador', 'laos', 'taiwan', 'haiti', 'portugal',
       'dominican-republic', 'el-salvador', 'france', 'guatemala',
       'china', 'japan', 'yugoslavia', 'peru',
       'outlying-us(guam-usvi-etc)', 'scotland', 'trinadad&tobago',
       'greece', 'nicaragua', 'vietnam', 'hong', 'ireland', 'hungary',
       'holand-netherlands'], dtype=object)

In [40]:
adult_df.loc[:,'native_region'] = adult_df['native_country'].map({
    'united-states': 'north america',
    'cambodia': 'asia',
    'england': 'europe',
    'puerto-rico': 'north america',
    'canada': 'north america',
    'germany': 'europe',
    'outlying-us(guam-usvi-etc)': 'north america',
    'india': 'asia',
    'japan': 'asia',
    'greece': 'europe',
    'south': 'south america',
    'china': 'asia',
    'cuba': 'central america',
    'iran': 'asia',
    'honduras': 'central america',
    'philippines': 'asia' ,
    'italy': 'europe' ,
    'poland': 'europe' ,
    'jamaica': 'central america' ,
    'vietnam': 'asia' ,
    'mexico': 'north america',
    'portugal': 'europe' ,
    'ireland': 'europe' ,
    'france': 'europe' ,
    'dominican-republic': 'central america' ,
    'laos': 'asia' ,
    'ecuador': 'south america',
    'taiwan': 'asia',
    'haiti': 'central america',
    'columbia': 'south america',
    'hungary': 'europe',
    'guatemala': 'central america',
    'nicaragua': 'central america',
    'scotland': 'europe',
    'thailand': 'asia',
    'yugoslavia': 'europe',
    'el-salvador': 'central america',
    'trinadad&tobago': 'central america',
    'peru': 'south america',
    'hong': 'asia',
    'other': 'other',
    'holand-netherlands': 'europe'

})

In [41]:
adult_df.columns

Index(['age', 'workclass', 'fnlwgt', 'education', ' education_num',
       'martial_status', 'occupation', 'relationship', 'race', 'sex',
       'capital_gain', 'capital_loss', 'hours_per_week', 'native_country',
       'income', 'education_level', 'occupation_grouped', 'native_region'],
      dtype='object')

In [42]:
adult_df['native_region'].unique()

array(['north america', 'central america', 'asia', nan, 'south america',
       'europe'], dtype=object)

In [43]:
adult_df

,age,workclass,fnlwgt,education,education_num,martial_status,occupation,relationship,race,sex,capital_gain,capital_loss,hours_per_week,native_country,income,education_level,occupation_grouped,native_region
0,39,state-gov,77516,bachelors,13,single,adm-clerical,single,white,male,2174,0,40,united-states,<=50k,tertiary,white collar,north america
1,50,self-emp-not-inc,83311,bachelors,13,married,exec-managerial,male spouse,white,male,0,0,13,united-states,<=50k,tertiary,white collar,north america
2,38,private,215646,hs-grad,9,divorced or separated,handlers-cleaners,single,white,male,0,0,40,united-states,<=50k,high school graduate,blue collar,north america
3,53,private,234721,11th,7,married,handlers-cleaners,male spouse,black,male,0,0,40,united-states,<=50k,secondary,blue collar,north america
4,28,private,338409,bachelors,13,married,prof-specialty,female spouse,black,female,0,0,40,cuba,<=50k,tertiary,white collar,central america
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32556,27,private,257302,assoc-acdm,12,married,tech-support,female spouse,white,female,0,0,38,united-states,<=50k,associate,white collar,north america
32557,40,private,154374,hs-grad,9,married,machine-op-inspct,male spouse,white,male,0,0,40,united-states,>50k,high school graduate,blue collar,north america
32558,58,private,151910,hs-grad,9,widowed,adm-clerical,single,white,female,0,0,40,united-states,<=50k,high school graduate,white collar,north america
32559,22,private,201490,hs-grad,9,single,adm-clerical,child,white,male,0,0,20,united-states,<=50k,high school graduate,white collar,north america


In [48]:
clean_filename = os.path.join(processed_dir, "adult_cleaned.csv")
adult_df.to_csv(clean_filename, index = False)
print(f"\nCleaned data saved to: {clean_filename}")


Cleaned data saved to: C:\Users\HP\Downloads\Adult_income\data\processed\adult_cleaned.csv


## Create age groups based on the age column

In [44]:
adult_df['age'].unique()

array([39, 50, 38, 53, 28, 37, 49, 52, 31, 42, 30, 23, 32, 40, 34, 25, 43,
       54, 35, 59, 56, 19, 20, 45, 22, 48, 21, 24, 57, 44, 41, 29, 18, 47,
       46, 36, 79, 27, 67, 33, 76, 17, 55, 61, 70, 64, 71, 68, 66, 51, 58,
       26, 60, 90, 75, 65, 77, 62, 63, 80, 72, 74, 69, 73, 81, 78, 88, 82,
       83, 84, 85, 86, 87], dtype=int64)

In [45]:
adult_df['age'].unique().min()

17

In [46]:
adult_df['age'].unique().max()

90

In [53]:
bins = [0, 18, 25, 35, 45, 60, 75, 100]
labels = ['<18', '18-25', '26-35', '36-45', '46-60', '61-75', '76+']
adult_df['age_group'] = pd.cut(adult_df['age'], bins = bins, labels = labels, right=True, include_lowest=True)

C:\Users\HP\AppData\Local\Temp\ipykernel_17748\343172918.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  adult_df['age_group'] = pd.cut(adult_df['age'], bins = bins, labels = labels, right=True, include_lowest=True)


In [55]:
adult_df['age_group'].unique()

['36-45', '46-60', '26-35', '18-25', '<18', '76+', '61-75']
Categories (7, object): ['<18' < '18-25' < '26-35' < '36-45' < '46-60' < '61-75' < '76+']

In [56]:
adult_df.drop(columns = ['education', 'native_country', 'occupation'], inplace=True)

KeyError: "['education', 'native_country', 'occupation'] not found in axis"

In [57]:
adult_df.columns

Index(['age', 'workclass', 'fnlwgt', ' education_num', 'martial_status',
       'relationship', 'race', 'sex', 'capital_gain', 'capital_loss',
       'hours_per_week', 'income', 'education_level', 'occupation_grouped',
       'native_region', 'age_group'],
      dtype='object')

In [58]:
adult_df.shape

(32537, 16)

In [59]:
adult_df.isnull().sum()

age                      0
workclass             1836
fnlwgt                   0
 education_num           0
martial_status           0
relationship             0
race                     0
sex                      0
capital_gain             0
capital_loss             0
hours_per_week           0
income                   0
education_level          0
occupation_grouped    1843
native_region          582
age_group                0
dtype: int64

In [60]:
adult_df.duplicated().sum()

23

In [61]:
adult_df[adult_df.duplicated(keep=False)]

,age,workclass,fnlwgt,education_num,martial_status,relationship,race,sex,capital_gain,capital_loss,hours_per_week,income,education_level,occupation_grouped,native_region,age_group
531,26,private,108658,9,single,single,white,male,0,0,40,<=50k,high school graduate,blue collar,north america,26-35
594,23,private,117789,13,single,child,white,female,0,0,40,<=50k,tertiary,white collar,north america,18-25
2896,46,private,271828,9,married,male spouse,white,male,0,0,40,>50k,high school graduate,blue collar,north america,46-60
3261,26,private,108658,9,single,single,white,male,0,0,40,<=50k,high school graduate,blue collar,north america,26-35
3586,28,private,50814,9,single,single,white,female,0,0,40,<=50k,high school graduate,white collar,north america,26-35
3692,46,private,271828,9,married,male spouse,white,male,0,0,40,>50k,high school graduate,blue collar,north america,46-60
3960,43,private,174575,10,divorced or separated,single,white,male,0,0,45,<=50k,some college,white collar,north america,36-45
4511,24,private,140001,13,single,single,white,male,0,0,40,<=50k,tertiary,white collar,north america,18-25
5110,21,private,118693,10,single,child,white,male,0,0,40,<=50k,some college,blue collar,north america,18-25
5805,20,private,107658,10,single,single,white,female,0,0,10,<=50k,some college,white collar,north america,18-25


In [63]:
adult_df = adult_df.drop_duplicates()

In [64]:
adult_df.duplicated().sum()

0

In [65]:
adult_df.shape

(32514, 16)

In [67]:
# Save the file in the processed data folder
final_file = os.path.join(processed_dir, 'adult_cleaned.csv')
adult_df.to_csv(final_file, index=False)

In [52]:
pd.cut([1,2,3], bins=[0, 2, 4], right=True )

[(0, 2], (0, 2], (2, 4]]
Categories (2, interval[int64, right]): [(0, 2] < (2, 4]]

In [53]:
pd.cut([1,2,3], bins=[0, 2, 4], right=False )

[[0, 2), [2, 4), [2, 4)]
Categories (2, interval[int64, left]): [[0, 2) < [2, 4)]

In [55]:
pd.cut([0,1,2], bins=[0, 1, 2], right=True, include_lowest=False )

[NaN, (0.0, 1.0], (1.0, 2.0]]
Categories (2, interval[int64, right]): [(0, 1] < (1, 2]]